# Building a custom hg38/WSN genome

My RNA seq examining 5'ppp RNA will return both human and influenza virus RNAs. I need to combine the human and WSN genomes so that I can align to both. This is more efficient than aligning to each separately and will give a better idea of coverage of both genomes.

## Which genome?

After much trial and error, use GENCODE's hg38 rel 46, as it is better annotated than RefSeq counterpart. Download primary assembly genome along with gff and gtf files.

## Which software

AGAT or gffread? Both are supposed to be good, maybe agat is more accurate but star is supposed to work with gffread. Using agat through these subsequent steps has resulted in star having the wrong length for a few transcripts and salmon can't quantify them because of the discrepancy in the .AlignedtoTranscriptome.bam file and the reference transcriptome .fasta. So use gffread I guess.

EDIT: gffread was stripping attributes out of my original file, causing agat to add back in duplicates to correct, this may be why agat wasn't working to generate a star compatible gtf. I will test this.

## Annotation

The human genome is well annotated and hg38 comes with a GTF and GFF file. STAR needs a gtf file when creating its index to figure out where transcripts are on the genome if we want to output additional transcriptome alignments. I need to make one for flu. Following sequence ontology rules, I'll make a GFF file as this is more feature rich and can be paired down to a gtf file by agat or gffread.

Once the gff file is made, check it using both agat and gffread for integrity and write output to compare. Agat adds rna features which are just transcripts, manually remove these or they'll show up in transcript file. Let's use gffread to convert to gtf since agat is annoying. **wait, see edit in software section** (I have tested agat for this and prefer gffread for now because it seems to work with star and agat edits stuff and may break star compatible formatting despite maybe being more true to gff or whatever it's been two days on this and I'm tired).



In [ ]:
gffread -E genome/WSN_annotated.gff > genome/WSN_annotated_test.gff
gffread -T genome/WSN_annotated.gff -o genome/WSN_annotated_gffread.gtf

EDIT: Wait, this is stripping some parental relationships for some reason leading to an incomplete gtf and causing a break in the chain for HA and NA gene -> transcript -> exon -> CDS. Try agat instead to create a finalized version of WSN_annotated. Results: it works and doesn't add randome extra HA/NA genes/transcripts anymore. Agat conversion to gtf preserves UTRs and genes in compliance with GTF3 guidlines but gffread throws these features out. Not sure which one star will take so I'll try to create an index out of each. If agat-derived gtf works we'll go with it.

Sanity check, processing the gencode gtf for the human genome lightly edits it to fold stop codons into CDS and modify wrong UTRs/remove supernumeries. It also removed 50 identical isoforms which will likely improve mapping.

Okay, gtf from flu is good, we also have gtf from hg38 via GENCODE. Let's combine them. Cat doesn't work by itself, transcripts extracted later only correspond to the first cat file (either WSN or human). We can combine with either agat or gffread. Agat has given me about a day's worth of problems trying to match it up to the transcriptome and star prefers gffread so let's try that.

In [ ]:
gffcompare genome/WSN_annotated_gffread.gtf \
    genome/gencode.v46.primary_assembly.annotation.gtf \
    -o genome/hybrid_annotated.gtf
gffread -w genome/hybrid_transcripts.fasta -g genome/hybrid.genome.fasta -gtf genome/hybrid_annotated.gtf

BIG PROBLEM: merging this way causes transcripts to be **RENAMED** by gffcompare to TCONS:x instead of their original names, therefore extracted transcripts are called TCONS and salmon can't quantify these from star later because names don't match up (I checked the docs for gffcompare and renaming transcripts to TCON: is normal behavior, let's not use this in this manner).

Looks like the problem is in the merge step. Let's instead cat the two together (which won't work alone) and then reformat using gffread.

In [ ]:
cat genome/WSN_annotated_gffread.gtf \
    genome/gencode.v46.primary_assembly.annotation.gtf \
    > genome/hybrid_annotated.gtf
gffread -E genome/hybrid_annotated.gtf -o genome/hybrid_annotated_test.gff
gffread -w genome/hybrid_transcripts.fasta -g genome/hybrid.genome.fasta -gtf genome/hybrid_annotated_test.gtf

Use the agat created genome from the edit to make star index and see if this works. When creating index, agat genome throws 0 errors or warnings. Next, will use to create a transcripts file by extracting sequences from the genome using the gtf as a guide. Won't use the gff as star needs the gtf and we want to eliminate any possible naming issues.